In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import cvxpy
import scipy
import scs
import superscs
from wurlitzer import pipes

In [2]:
print(scs.__version__)
assert(int(scs.__version__[0]) >= 2)

2.0.2


In [3]:
probs = []

In [4]:
# Random LP
np.random.seed(hash('lp') % 2 ** 31)

# Dimensions
n = 100
m = 70

A = scipy.sparse.random(m, n, density=0.2, data_rvs = np.random.randn)
b = np.random.randn(m, 1)
c = np.random.rand(n, 1)

# Problem construction
x = cvxpy.Variable(n)
objective = cvxpy.Minimize(c.T * x)
constraints = [x >= 0, A * x <= b]
probs.append(cvxpy.Problem(objective, constraints))

In [5]:
# Optimal control
np.random.seed(hash('opt-control') % 2 ** 31)

# Problem data
T = 10
n, p = (10, 5)
A = np.random.randn(n, n)
B = np.random.randn(n, p)
x_init = np.random.randn(n, 1)
x_final = np.random.randn(n, 1)

def step(A, B, x_prev):
    x = cvxpy.Variable(n, 1)
    u = cvxpy.Variable(p, 1)
    cost = sum(cvxpy.square(u)) + sum(cvxpy.abs(x))
    constraint = (x == A * x_prev + B * u)
    return cost, constraint, x

x = cvxpy.Variable(n, 1)
constraints = [(x == x_init)]
total_cost = 0.
for t in range(T):
    cost_t, constraint_t, x = step(A, B, x)
    constraints.append(constraint_t)
    total_cost += cost_t

constraints.append(x == x_final)
probs.append(cvxpy.Problem(cvxpy.Minimize(total_cost), constraints))

In [6]:
# Lasso
np.random.seed(hash('lasso') % 2 ** 31)

# Dimensions
n = 100
m = 50

x_true = scipy.sparse.rand(n, 1, density=0.1) 
A = scipy.sparse.random(m, n, density=0.2, data_rvs = np.random.randn)
b = A * x_true + 0.1 * np.random.randn(m, 1)
mu = 1

# Problem construction
x = cvxpy.Variable(n)
objective = cvxpy.Minimize(0.5 * cvxpy.sum_squares(A*x - b) + mu * cvxpy.norm1(x))
probs.append(cvxpy.Problem(objective))

In [7]:
# Nonnegative Lasso
np.random.seed(hash('nonneg-lasso') % 2 ** 31)

# Dimensions
n = 100
m = 50

x_true = scipy.sparse.rand(n, 1, density=0.1) 
A = scipy.sparse.random(m, n, density=0.2, data_rvs = np.random.randn)
b = A * x_true + 0.1 * np.random.randn(m, 1)
mu = 1

# Problem construction
x = cvxpy.Variable(n)
objective = cvxpy.Minimize(0.5 * cvxpy.sum_squares(A*x - b) + mu * cvxpy.norm1(x))
constraints = [x >= 0]
probs.append(cvxpy.Problem(objective, constraints))

In [8]:
# SDP for closest elemwise-positive PSD matrix in some metric
np.random.seed(hash('sdp') % 2 ** 31)

# create data P
n = 50
P = np.random.randn(n, n)
P = P + P.T
Z = cvxpy.semidefinite(n)

objective = cvxpy.Maximize(cvxpy.lambda_min(P - Z))
probs.append(cvxpy.Problem(objective, [Z >= 0]))

In [9]:
# Basis pursuit
np.random.seed(hash('basis-pursuit') % 2 ** 31)

n = 300
m = 100
x = cvxpy.Variable(n)
A = np.random.rand(m, n)
x0 = scipy.sparse.rand(n, 1, 0.1)
b = A*x0

probs.append(cvxpy.Problem(cvxpy.Minimize(cvxpy.norm1(x)), [A*x == b]))

In [10]:
# Chebyshev
np.random.seed(hash('chebyshev') % 2 ** 31)

def normalized_data_matrix(m, n):
    A = np.random.randn(m, n)
    A /= np.sqrt(np.sum(A**2, 0))
    return A

m = 100
n = 200
k = 50
A = [normalized_data_matrix(m, n) for i in range(k)]
B = normalized_data_matrix(k, n)
c = np.random.rand(k)

x = cvxpy.Variable(n)
t = cvxpy.Variable(k)

f = cvxpy.max_entries(t + cvxpy.abs(B * x - c))
constraints = []
for i in range(k):
    constraints.append(cvxpy.norm(A[i]*x) <= t[i])

probs.append(cvxpy.Problem(cvxpy.Minimize(f), constraints))

In [11]:
# Least absolute deviation
np.random.seed(hash('least-abs-dev') % 2 ** 31)

m = 5000
n = 200

A = np.random.randn(m,n);
b = A.dot(np.random.randn(n))

# Add outlier noise.
k = int(0.02 * m)
idx = np.random.randint(m, size=k)
b[idx] += 10 * np.random.randn(k)

x = cvxpy.Variable(n)

probs.append(cvxpy.Problem(cvxpy.Minimize(cvxpy.norm1(A*x - b))))

In [12]:
# P-norm
np.random.seed(hash('p-norm') % 2 ** 31)

n = 20
m = int(n / 4)

G = np.random.randn(m, n)
f = np.random.randn(m, 1)

power = np.pi
x = cvxpy.Variable(n)
constraints = [G*x == f]
probs.append(cvxpy.Problem(cvxpy.Minimize(cvxpy.norm(x, power)), constraints))

In [13]:
# L1-regularized Logistic regression
np.random.seed(hash('log-reg') % 2 ** 31)

p = 1000   # features
q = 10 * p  # total samples

w_true = np.random.randn(p, 1)
X_tmp = np.random.randn(p, q)

ips = -w_true.T.dot(X_tmp)
ps = (np.exp(ips)/(1 + np.exp(ips))).T
labels = 2*(np.random.rand(q,1) < ps) - 1

X_pos = X_tmp[:,np.where(labels==1)[0]]
X_neg = X_tmp[:,np.where(labels==-1)[0]]
X = np.hstack([X_pos, -X_neg]) # include labels with data

lam = 2
w = cvxpy.Variable(p, 1)
obj = (cvxpy.sum_entries(cvxpy.log_sum_exp(cvxpy.vstack([np.zeros((1,q)), w.T*X]), axis = 0))
        + lam * cvxpy.norm(w,1))

probs.append(cvxpy.Problem(cvxpy.Minimize(obj)))

In [14]:
# Matrix completion
np.random.seed(hash('matrix-completion') % 2 ** 31)

m = 100
n = 50

M = np.random.randn(m * n)
n_missing = int(0.8 * m * n)
missing_idx = np.random.permutation(m * n)[:n_missing]
valid_idx = np.setdiff1d(np.arange(m * n), missing_idx)

M[missing_idx] = 0.
X = cvxpy.Variable(m * n)

lam = 0.5
diff = cvxpy.reshape(X, m, n) - np.reshape(M, (m, n))
obj = cvxpy.norm(diff, "nuc") + lam * cvxpy.sum_squares(X)
constraints = [X[valid_idx] == M[valid_idx]]

probs.append(cvxpy.Problem(cvxpy.Minimize(obj), constraints))

In [15]:
# Min-norm
np.random.seed(hash('min-norm') % 2 ** 31)

m = 500
n = int(m / 2)
A = np.random.randn(m, n)
b = 10 * np.random.randn(m, 1)
G = 2 * np.random.randn(2 * n, n)

x = cvxpy.Variable(n)
obj = cvxpy.norm(A * x - b)
constraints = [cvxpy.norm(G * x) <= 1]

probs.append(cvxpy.Problem(cvxpy.Minimize(obj), constraints))

In [16]:
# Lyapunov stability
np.random.seed(hash('lyapunov') % 2 ** 31)

n = 100
A = np.diag(-np.logspace(-0.5, 1, n))
U = scipy.linalg.orth(np.random.randn(n,n))
A = U.T.dot(A.dot(U))

P = cvxpy.Symmetric(n, n)
obj = cvxpy.trace(P)
constraints = [A.T*P + P*A << -np.eye(n), P >> np.eye(n)]

probs.append(cvxpy.Problem(cvxpy.Minimize(obj), constraints))

In [17]:
# Portfolio
np.random.seed(hash('portfolio') % 2 ** 31)

m = 500
n = 5000
density = 0.1

mu = np.exp(0.01 * np.random.randn(n)) - 1.  # returns
D = np.random.rand(n) / 10.  # idiosyncratic risk
F = scipy.sparse.rand(n, m, density) / 10.  # factor model

lambda_risk = 1
leverage = 1

x = cvxpy.Variable(n)
obj = mu.T * x - lambda_risk * (cvxpy.sum_squares(F.T.dot(x)) +
                              cvxpy.sum_squares(cvxpy.mul_elemwise(D, x)))

constraints = [cvxpy.sum_entries(x) == leverage, x >= 0]

probs.append(cvxpy.Problem(cvxpy.Maximize(obj), constraints))

In [18]:
# Sparse covariance estimation
np.random.seed(hash('cov-estim') % 2 ** 31)

num_samples = 10
n = 20
lam = 0.1

A = scipy.sparse.rand(n, n, 0.01)
A = A.T.dot(A).todense() + 0.1 * np.eye(n)
L = np.linalg.cholesky(np.linalg.inv(A))
X = L.dot(np.random.randn(n, num_samples)) # Draw m experiments according to the covariance matrix A^-1
S = X.dot(X.T) / num_samples # Estimate of covariance matrix
mask = np.ones((n,n)) - np.eye(n)

theta = cvxpy.Variable(n, n)

obj = (lam*cvxpy.norm1(cvxpy.mul_elemwise(mask, theta)) +
       cvxpy.trace(S * theta) - cvxpy.log_det(theta))
probs.append(cvxpy.Problem(cvxpy.Minimize(obj)))

In [19]:
# Fused Lasso
np.random.seed(hash('fused-lasso') % 2 ** 31)

m = 1000
ni = 10
k = 1000
rho=0.05
sigma=0.05

A = np.random.randn(m, ni * k)
A /= np.sqrt(np.sum(A ** 2, 0))

x0 = np.zeros(ni * k)
for i in range(k):
    if np.random.rand() < rho:
        x0[i * ni:(i + 1) * ni] = np.random.rand()

b = A.dot(x0) + sigma * np.random.randn(m)
lam = 0.1 * sigma * np.sqrt(m * np.log(ni * k))

x = cvxpy.Variable(ni * k)
obj = cvxpy.sum_squares(A * x - b) + lam * cvxpy.norm1(x) + lam * cvxpy.tv(x)
probs.append(cvxpy.Problem(cvxpy.Minimize(obj)))

In [20]:
# SVM
np.random.seed(hash('svm') % 2 ** 31)

m = 150
n = 500
A = np.random.randn(m, n)
x0 = np.random.rand(n)
y = np.sign(A.dot(x0) + 0.05*np.random.randn(m))

lam = 1.0
x = cvxpy.Variable(n)
obj = ((1./m) * cvxpy.sum_entries(cvxpy.pos(1 - cvxpy.mul_elemwise(y, A * x)))
       + lam * cvxpy.norm(x, 1))

probs.append(cvxpy.Problem(cvxpy.Minimize(obj)))

In [21]:
# Robust PCA
np.random.seed(hash('robust-pca') % 2 ** 31)

n = 100
r = 10 # Rank
density = 0.1

L0 = np.random.randn(n, r).dot(np.random.randn(r, n)) # Low rank matrix
S0 = scipy.sparse.rand(n, n, density) # Sparse matrix w/ Normally distributed entries.
S0.data = 10 * np.random.randn(len(S0.data))

M = L0 + S0

L = cvxpy.Variable(n, n)
S = cvxpy.Variable(n, n)

lam = 0.1
obj = cvxpy.norm(L, "nuc") + lam * cvxpy.norm1(S)
constraints = [L + S == M]

probs.append(cvxpy.Problem(cvxpy.Minimize(obj), constraints))

In [22]:
# Robust Gaussian covariance estimation
np.random.seed(hash('rob-cov-var') % 2 ** 31)

m = 11 # Number of observations of each random variable
n = 5 # Number of random variables
k = 3 # Needs to be less than m. 

A = np.matrix(np.random.rand(m, n))
A -= np.mean(A, axis=0)
K = np.array([(A[i].T * A[i]).flatten() for i in range(m)])

sigma_inv1 = cvxpy.Variable(n,n) # Inverse covariance matrix
t = cvxpy.Variable(m)
tdet = cvxpy.Variable(1)

obj = cvxpy.sum_largest(t+tdet, k)
z = K*cvxpy.reshape(sigma_inv1, n*n, 1)
constraints = [-cvxpy.log_det(sigma_inv1) <= tdet, t == z]
probs.append(cvxpy.Problem(cvxpy.Minimize(obj), constraints))

In [23]:
# Infinite push
np.random.seed(hash('infinite-push') % 2 ** 31)

m = 100
n = 200
d = 20
np.random.seed(0)

Xp = np.random.randn(m, d)
Xn = np.random.randn(n, d)

lam = 1
theta = cvxpy.Variable(d)
Z = cvxpy.max_elemwise(1 - (Xp * theta * np.ones((1,n)) - (Xn * theta * np.ones((1,m))).T), 0)
obj = cvxpy.max_entries(cvxpy.sum_entries(Z, axis=0)) + lam * cvxpy.sum_squares(theta)
probs.append(cvxpy.Problem(cvxpy.Minimize(obj)))

In [24]:
# Quantile regression
np.random.seed(hash('quantile-regression') % 2 ** 31)

m = 100 # Number of data entries
n = 5 # Number of weights
k = 20 # Number of quantiles
p = 1
sigma = 0.1

x = np.random.rand(m)* 2 * np.pi * p
y = np.sin(x) + sigma * np.sin(x) * np.random.randn(m)
alphas = np.linspace(1. / (k + 1), 1 - 1. / (k + 1), k) # Do a bunch of quantiles at once

# RBF (Radial Basis Function) features
mu_rbf = np.array([np.linspace(-1, 2 * np.pi * p + 1, n)])
mu_sig = (2 * np.pi * p + 2)/n
X = np.exp(-(mu_rbf.T - x).T**2 / (2 * mu_sig**2)) # Gaussian

theta = cvxpy.Variable(n,k)

def quantile_loss(alphas, theta, X, y):
    m, n = X.shape
    k = len(alphas)
    Y = np.tile(y.flatten(), (k, 1)).T
    A = np.tile(alphas, (m, 1))
    Z = X * theta - Y
    return cvxpy.sum_entries(
        cvxpy.max_elemwise(
            cvxpy.mul_elemwise(-A, Z),
            cvxpy.mul_elemwise(1 - A, Z)))

obj = quantile_loss(alphas, theta, X, y)
constraints = [X*(theta[:,1:] - theta[:,:-1]) >= 0]
probs.append(cvxpy.Problem(cvxpy.Minimize(obj), constraints))

In [25]:
# Huber regression
np.random.seed(hash('huber-regression') % 2 ** 31)

m = 5000
n = 200

x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A.dot(x0) + 0.01 * np.random.randn(m)
# Add outlier noise.
k = int(0.02 * m)
idx = np.random.randint(m, size=k)
b[idx] += 10 * np.random.randn(k)

x = cvxpy.Variable(n)
probs.append(cvxpy.Problem(cvxpy.Minimize(cvxpy.sum_entries(cvxpy.huber(A*x - b)))))

In [ ]:
stgs = {
    'use_indirect' : True,
    'eps' : 1e-5,
    'verbose' : True,
    'max_iters' : 5000,
}
acceleration_lookback = 20

"""
        /**
         * Restarted Broyden method
         */
        restarted_broyden = 100,
        /**
         * Restarted Broyden method (experimental)
         */
        restarted_broyden_v2 = 101,
        /**
         * Anderson's acceleration
         */
        anderson_acceleration = 150,
        /**
         * Using \f$d_k = - R_k\f$
         */
        fixed_point_residual = 200,
        /**
         * Full Broyden method
         */
        full_broyden = 300
"""


scs_times = []
superscs_times = []
scs_iters = []
superscs_iters = []
for prob in probs:

    print('****************************************************************************')
    print('****************************************************************************')

    # For some reason I can't get CVXPY to recognize SuperSCS is installed.
    d = prob.get_problem_data(solver='SCS')
    scs.solve(d, d['dims'], acceleration_lookback=acceleration_lookback, **stgs)
    # SuperSCS is printing to the terminal instead of the notebook
    # cell, so this useful package captures it and I print it out.
    with pipes() as (out, err):
        superscs.solve(d, d['dims'], memory=acceleration_lookback, direction=100, **stgs)
        superscs.solve(d, d['dims'], memory=acceleration_lookback, direction=101, **stgs)
        superscs.solve(d, d['dims'], memory=acceleration_lookback, direction=150, **stgs)
        superscs.solve(d, d['dims'], memory=acceleration_lookback, direction=200, **stgs)
        superscs.solve(d, d['dims'], memory=acceleration_lookback, direction=300, **stgs)
    print(out.read())

****************************************************************************
****************************************************************************
----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1500, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 100, constraints m = 170
Cones:	linear vars: 170
Setup time: 7.03e-03s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.09e+20  1.21e+20  1.00e+00 -6.79e+20  2.79e+20  1.95e+20  8.99e-03 
   100| 5.66e-03  

---------------------------------------------------------------------------------------
	SCS v1.2.6-KUL-SuperMann - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
---------------------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 2170, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
direction = 100, memory = 20
Variables n = 310, constraints m = 470
Cones:	primal zero / dual free vars: 120
	linear vars: 200
	soc vars: 150, soc blks: 50
Setup time: 6.59e-05s
Running SuperSCS...
Memory: 477.10kB
---------------------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau |   FPR   | time (s)
---------------------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  1.77e+01  1.50e-04 
   100| 1

---------------------------------------------------------------------------------------
	SCS v1.2.6-KUL-SuperMann - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
---------------------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1402, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
direction = 100, memory = 20
Variables n = 201, constraints m = 252
Cones:	linear vars: 200
	soc vars: 52, soc blks: 1
Setup time: 9.42e-05s
Running SuperSCS...
Memory: 282.10kB
---------------------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau |   FPR   | time (s)
---------------------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  1.44e+01  1.92e-04 
    80| 1.03e-06  2.00e-06  7.44e-07  5.33e+00

---------------------------------------------------------------------------------------
	SCS v1.2.6-KUL-SuperMann - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
---------------------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1502, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
direction = 100, memory = 20
Variables n = 201, constraints m = 352
Cones:	linear vars: 300
	soc vars: 52, soc blks: 1
Setup time: 8.30e-05s
Running SuperSCS...
Memory: 336.50kB
---------------------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau |   FPR   | time (s)
---------------------------------------------------------------------------------------
     0|      inf       inf       nan      -inf       inf       inf  1.61e+01  1.45e-04 
    60| 6.08e-07  4.38e-06  3.71e-07  5.18e+00

---------------------------------------------------------------------------------------
	SCS v1.2.6-KUL-SuperMann - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
---------------------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 6325, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
direction = 100, memory = 20
Variables n = 1276, constraints m = 6275
Cones:	primal zero / dual free vars: 1225
	linear vars: 2500
	sd vars: 2550, sd blks: 2
Setup time: 6.96e-04s
Running SuperSCS...
Memory: 4.15MB
---------------------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau |   FPR   | time (s)
---------------------------------------------------------------------------------------
     0| 5.12e+01  4.31e+01  9.99e-01 -3.57e+02  8.35e+02  0.00e+00  6.10e+01  1.60e-03 
   100| 1

---------------------------------------------------------------------------------------
	SCS v1.2.6-KUL-SuperMann - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
---------------------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 31200, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
direction = 100, memory = 20
Variables n = 600, constraints m = 700
Cones:	primal zero / dual free vars: 100
	linear vars: 600
Setup time: 4.65e-04s
Running SuperSCS...
Memory: 1.68MB
---------------------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau |   FPR   | time (s)
---------------------------------------------------------------------------------------
     0| 1.50e+01  1.45e+01  9.99e-01 -1.86e+03  4.85e+01  0.00e+00  2.47e+01  1.40e-03 
   100| 1.46e-04  2.55e-04  1.11e-04  1

----------------------------------------------------------------------------
	SCS v2.0.2 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2017
----------------------------------------------------------------------------
Lin-sys: sparse-indirect, nnz in A = 1020400, CG tol ~ 1/iter^(2.00)
eps = 1.00e-05, alpha = 1.50, max_iters = 5000, normalize = 1, scale = 1.00
acceleration_lookback = 20, rho_x = 1.00e-03
Variables n = 351, constraints m = 5250
Cones:	linear vars: 200
	soc vars: 5050, soc blks: 50
Setup time: 4.29e-02s
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 4.42e+20  4.17e+20  1.00e+00 -2.33e+20  1.23e+21  1.15e+20  1.12e-01 
   100| 1.74e-03  4.07e-03  5.03e-04  9.60e-01  9.58e-01  1.15e-16  3.11e+00 
   200| 7.39e-06  1.95e-05  5.36e-07  9.59e-01  9.59e-01  5.